In [1]:
import torch
import numpy as np

import random
import os
import pickle
import matplotlib.pyplot as plt
import copy


def seed_torch(RANDOM_SEED=123):
    random.seed(RANDOM_SEED)
    os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_torch()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:',device)

from utils import generate_series, temporal_split
from model import HOIST_without_claim
from utils import mse,mae,r2,ccc


device: cuda


In [3]:
mob_mat = pickle.load(open('./data/mob_mat.pkl', 'rb'))
distance_mat = pickle.load(open('./data/distance_mat.pkl', 'rb'))
covid_tensor = pickle.load(open('./data/covid_tensor.pkl', 'rb'))
hospitalizations = pickle.load(open('./data/hospitalizations.pkl', 'rb'))
hos_tensor = pickle.load(open('./data/hos_tensor.pkl', 'rb'))
county_tensor = pickle.load(open('./data/county_tensor.pkl', 'rb'))
feat_name = pickle.load(open('./data/feat_name.pkl', 'rb'))
date_range = np.array(pickle.load(open('./data/date_range.pkl', 'rb')))

In [4]:
print("mob_mat shape:", mob_mat.shape)
print("distance_mat shape:", distance_mat.shape)
print("covid_tensor shape:", covid_tensor.shape)
print("hospitalizations shape:", hospitalizations.shape)
print("hos_tensor shape:", hos_tensor.shape)
print("county_tensor shape:", county_tensor.shape)
print("feat_name:", feat_name)
print("date_range shape:", date_range.shape)


mob_mat shape: (2334, 2334)
distance_mat shape: (2334, 2334)
covid_tensor shape: (2334, 639)
hospitalizations shape: (2334, 639)
hos_tensor shape: (2334, 639, 4)
county_tensor shape: (2334, 14)
feat_name: {'hospitalization': ['inbeds', 'inbeds_covid', 'icu', 'icu_covid'], 'vaccination': ['1st', '2nd', 'bst', 'Pfizer_1', 'Pfizer_2', 'Pfizer_b', 'Moderna_1', 'Moderna_2', 'Moderna_b', 'Johnson_1', 'Johnson_b', 'PfizerTS_1', 'PfizerTS_2', 'PfizerTS_b', 'PfizerTS10_1', 'PfizerTS10_2'], 'claim': ['hospitalization', 'n_visits', 'age_cnt', 'Cerebrovascular Disease', 'Chronic Pulmonary Disease', 'Congestive Heart Failure', 'Dementia', 'Diabetes without chronic complication', 'HIV', 'Hemiplegia or Paraplegia', 'Hypertension', 'Immunodeficiency', 'Liver Disease', 'Malignancy', 'Metastatic Solid Tumor', 'Myocardial Infarction', 'Obesity', 'Peptic Ulcer Disease', 'Peripheral Vascular Disease', 'Renal'], 'county': ['pop', '0_17', '18_64', '65p', 'Black', 'White', 'Asian', 'Hispanic', 'Not_Hispanic',

In [5]:
# Align regions across datasets
# Find common regions across datasets
regions_covid = set(range(covid_tensor.shape[0]))


In [6]:
regions_static = set(range(county_tensor.shape[0]))

In [7]:
print("hospitalizations shape: ", hospitalizations.shape)

hospitalizations shape:  (2334, 639)


In [8]:
print("hos_tensor shape: ", hos_tensor.shape)

hos_tensor shape:  (2334, 639, 4)


# Temporal split

In [9]:
covid_tensor = np.expand_dims(covid_tensor, axis=2)
X = np.concatenate([covid_tensor, hos_tensor], axis=2)
y = hospitalizations
X, y = generate_series(X, y, window_size=35, pred_size=28)
date_idx = np.expand_dims(date_range, axis=0)
date_idx = np.expand_dims(date_idx, axis=2)
date_idx, _ = generate_series(date_idx, y, window_size=35, pred_size=28, date=True)

range_idx = (y.mean(1)>0)
county_tensor = county_tensor[range_idx]
y = y[range_idx]
X = X[range_idx]
print(len(y))
mob_mat = mob_mat[range_idx, :][:, range_idx]
distance_mat = distance_mat[range_idx, :][:, range_idx]

y = np.log(y+1)
train_x, val_x, test_x, train_y, val_y, test_y, train_idx, val_idx, test_idx, static, mats, normalize_dict, shuffle_idx = temporal_split(X, y, county_tensor, [mob_mat, distance_mat], 0.2, 0.2, norm='min-max', norm_mat=True)

norm_mob = mats[0]
norm_dist = mats[1]

# print the shape of the training data
print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)
print("val_x shape:", val_x.shape)
print("val_y shape:", val_y.shape)


2299
train_x shape: (2299, 13, 5)
train_y shape: (2299, 13)
val_x shape: (2299, 4, 5)
val_y shape: (2299, 4)


In [10]:
mae_ = []
mae_exp = []
mse_ = []
mse_exp = []
r2_ = []
r2_exp = []
ccc_ = []
ccc_exp = []

runs = 5
for k in range(runs):
    seed_torch(k)
    model = HOIST_without_claim(5, [4,5,5], 128, device).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    loss_fn = torch.nn.MSELoss(reduction='none')

    epoch = 300
    batch_size = 128
    min_loss = 1e99
    min_epoch = 0

    for i in range(epoch):
        epoch_loss = []
        val_loss = []
        model.train()
        for j in range((len(test_x)//batch_size)+1):
            batch_x = train_x[j*batch_size:(j+1)*batch_size]
            batch_y = train_y[j*batch_size:(j+1)*batch_size]
            batch_x = torch.tensor(batch_x).float().to(device)
            batch_y = torch.tensor(batch_y).float().to(device).unsqueeze(-1)
            batch_static = torch.tensor(static[j*batch_size:(j+1)*batch_size]).float().to(device)
            batch_mob = torch.tensor(norm_mob[j*batch_size:(j+1)*batch_size,:][:,j*batch_size:(j+1)*batch_size]).float().to(device)
            batch_dist = torch.tensor(norm_dist[j*batch_size:(j+1)*batch_size,:][:,j*batch_size:(j+1)*batch_size]).float().to(device)
            batch_mat = torch.cat([batch_mob.unsqueeze(-1), batch_dist.unsqueeze(-1)], dim=2)
            cur_static = [batch_static[:, :4], batch_static[:, 4:9], batch_static[:, 9:14], batch_mat]
            
            optimizer.zero_grad()
            output, _ = model(batch_x, cur_static)
            
            N, T, F = batch_y.shape
            dist = _[0]
            weights = _[1]
            y_p = (weights * batch_x).sum(-1).reshape(N,T,1)*output.detach()
            y_pi = y_p.reshape(N,1,T)
            y_pj = y_p.reshape(1,N,T)
            y_k = ((y_pi * y_pj) * dist.reshape(N,N,1)).sum(1).reshape(N,T,1)
            ising_loss = loss_fn(y_p+y_k, batch_y).mean(1).mean()
            
            loss = loss_fn(output, batch_y).mean(1).mean() + ising_loss
            loss.backward()
            optimizer.step()
            epoch_loss.append(loss.item())
        
        model.eval()
        y_pred = []
        y_true = []
        with torch.no_grad():
            for j in range((len(test_x)//batch_size)+1):
                batch_x = val_x[j*batch_size:(j+1)*batch_size]
                batch_y = val_y[j*batch_size:(j+1)*batch_size]
                batch_x = torch.tensor(batch_x).float().to(device)
                batch_y = torch.tensor(batch_y).float().to(device).unsqueeze(-1)
                batch_static = torch.tensor(static[j*batch_size:(j+1)*batch_size]).float().to(device)
                batch_mob = torch.tensor(norm_mob[j*batch_size:(j+1)*batch_size,:][:,j*batch_size:(j+1)*batch_size]).float().to(device)
                batch_dist = torch.tensor(norm_dist[j*batch_size:(j+1)*batch_size,:][:,j*batch_size:(j+1)*batch_size]).float().to(device)
                batch_mat = torch.cat([batch_mob.unsqueeze(-1), batch_dist.unsqueeze(-1)], dim=2)
                cur_static = [batch_static[:, :4], batch_static[:, 4:9], batch_static[:, 9:14], batch_mat]
                
                output, _ = model(batch_x, cur_static)
                loss = loss_fn(output, batch_y).mean(1).mean()
                y_pred += list(output.squeeze().cpu().detach().numpy())
                y_true += list(batch_y.squeeze().cpu().detach().numpy())
                val_loss.append(loss.item())
        y_pred = np.array(y_pred)
        y_true = np.array(y_true)
        norm_pred = (y_pred * normalize_dict['y'][1]) + normalize_dict['y'][0]
        norm_true = (y_true * normalize_dict['y'][1]) + normalize_dict['y'][0]
        
        cur_mse = mse(norm_true, norm_pred)
        cur_mae = mae(norm_true, norm_pred)
        if i % 100 == 0:
            print('Epoch: %d, Train Loss: %.4f, Val Loss: %.4f, MSE: %.2f, MAE: %.2f'%(i, np.mean(epoch_loss), np.mean(val_loss), cur_mse, cur_mae))
        if cur_mae < min_loss:
            min_loss = cur_mae
            min_epoch = i
            torch.save(model.state_dict(), './model/hoist_%d.pth'%k)
            
    y_pred = []
    y_true = []
    weight_score = []
    batch_size = 128
    #Load state dict
    model.load_state_dict(torch.load('./model/hoist_%d.pth'%k))
    model.eval()

    for j in range((len(test_x)//batch_size)+1):
        batch_x = test_x[j*batch_size:(j+1)*batch_size]
        batch_y = test_y[j*batch_size:(j+1)*batch_size]
        batch_x = torch.tensor(batch_x).float().to(device)
        batch_y = torch.tensor(batch_y).float().to(device).unsqueeze(-1)
        batch_static = torch.tensor(static[j*batch_size:(j+1)*batch_size]).float().to(device)
        batch_mob = torch.tensor(norm_mob[j*batch_size:(j+1)*batch_size,:][:,j*batch_size:(j+1)*batch_size]).float().to(device)
        batch_dist = torch.tensor(norm_dist[j*batch_size:(j+1)*batch_size,:][:,j*batch_size:(j+1)*batch_size]).float().to(device)
        batch_mat = torch.cat([batch_mob.unsqueeze(-1), batch_dist.unsqueeze(-1)], dim=2)
        cur_static = [batch_static[:, :4], batch_static[:, 4:9], batch_static[:, 9:14], batch_mat]
        output, _ = model(batch_x, cur_static)
        
        y_pred += list(output.squeeze().cpu().detach().numpy())
        y_true += list(batch_y.squeeze().cpu().detach().numpy())
        weight_score += list(_[1].squeeze().cpu().detach().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    weight_score = np.array(weight_score)


    norm_pred = (y_pred * normalize_dict['y'][1]) + normalize_dict['y'][0]
    norm_true = (y_true * normalize_dict['y'][1]) + normalize_dict['y'][0]
    
    print('Best Epoch: %d, Test MSE: %.2f, MAE: %.2f, R2: %.2f, CCC: %.2f'%(min_epoch, mse(norm_true, norm_pred), mae(norm_true, norm_pred), r2(norm_true, norm_pred), ccc(norm_true, norm_pred)))
    mae_.append(mae(norm_true, norm_pred))
    mae_exp.append(mae(np.exp(norm_true), np.exp(norm_pred)))
    mse_.append(mse(norm_true, norm_pred))
    mse_exp.append(mse(np.exp(norm_true), np.exp(norm_pred)))
    r2_.append(r2(norm_true, norm_pred))
    r2_exp.append(r2(np.exp(norm_true), np.exp(norm_pred)))
    ccc_.append(ccc(norm_true, norm_pred))
    ccc_exp.append(ccc(np.exp(norm_true), np.exp(norm_pred)))

Epoch: 0, Train Loss: 1.9760, Val Loss: 0.8685, MSE: 5.91, MAE: 2.06
Epoch: 100, Train Loss: 1.0821, Val Loss: 0.2432, MSE: 1.66, MAE: 1.00
Epoch: 200, Train Loss: 1.0708, Val Loss: 0.2465, MSE: 1.68, MAE: 1.00
Best Epoch: 92, Test MSE: 1.90, MAE: 1.08, R2: 0.73, CCC: 0.87
Epoch: 0, Train Loss: 1.9757, Val Loss: 0.9133, MSE: 6.22, MAE: 2.12
Epoch: 100, Train Loss: 1.0910, Val Loss: 0.2588, MSE: 1.76, MAE: 1.03
Epoch: 200, Train Loss: 1.0796, Val Loss: 0.2520, MSE: 1.72, MAE: 1.01
Best Epoch: 174, Test MSE: 1.92, MAE: 1.08, R2: 0.72, CCC: 0.87
Epoch: 0, Train Loss: 1.9735, Val Loss: 0.8579, MSE: 5.84, MAE: 2.05
Epoch: 100, Train Loss: 1.0910, Val Loss: 0.2436, MSE: 1.66, MAE: 1.00
Epoch: 200, Train Loss: 1.0805, Val Loss: 0.2613, MSE: 1.78, MAE: 1.04
Best Epoch: 105, Test MSE: 1.89, MAE: 1.08, R2: 0.73, CCC: 0.87
Epoch: 0, Train Loss: 1.9350, Val Loss: 0.8635, MSE: 5.88, MAE: 2.06
Epoch: 100, Train Loss: 1.0800, Val Loss: 0.2429, MSE: 1.65, MAE: 1.00
Epoch: 200, Train Loss: 1.0635, Val 

In [11]:
print(np.mean(mse_), np.std(mse_))
print(np.mean(mse_exp), np.std(mse_exp))
print(np.mean(mae_), np.std(mae_))
print(np.mean(mae_exp), np.std(mae_exp))
print(np.mean(r2_), np.std(r2_))
print(np.mean(r2_exp), np.std(r2_exp))
print(np.mean(ccc_), np.std(ccc_))
print(np.mean(ccc_exp), np.std(ccc_exp))

1.8987061 0.019217249
6811341.0 2027739.1
1.0797278 0.0028859947
614.2911 88.46489
0.7271939635276794 0.002738882621223715
0.4928076684474945 0.1761980838343089
0.8687882773864812 0.0017405191077176434
0.7790053094829344 0.037898322396380156
